<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF-280 - Estadística Computacional </h1>
    <h2> Test de hipótesis </h2>
    <h4> Diego Quezada </h4>
</center>

## Contenidos

* [Introducción](#intro)
* [Reglamento](#rules)
* [Experiencia](#experience)
    * [Teorema del límite central](#tcl)
    * [Prueba de hipótesis](#ht)
    * [Prueba de permutación](#pt)

<div id='intro' />

## Introducción

Los laboratorios de estadística computacional (LEC) tienen por objetivo principal analizar datos utilizando técnicas de visualización y evidenciar el comportamiento estocástico de experimentos aleatorios mediante simulaciones computacionales. Las experiencias buscan medir la habilidad de programación en Python y sus librerías, la capacidad de análisis estadístico y la comprensión de documentación, artículos y papers.

Recuerde que los laboratorios tienen una ponderación de 30% en la nota final del ramo.

<div id='rules' />

## Reglamento

1. El desarrollo de los laboratorios debe ser en **Python**.
2. El formato de entrega es un **archivo .ipynb**, es decir, un jupyter notebook.
3. El nombre del archivo de entrega del laboratorio $i$ debe seguir el siguiente formato: *lec-i-nombregrupo.ipynb*.
4. Se recomienda seguir las recomendaciones de estilo descritas en [PEP 8](https://www.python.org/dev/peps/pep-0008/) a la hora de programar.
5. El tiempo para la realización de los laboratorios es extenso, por lo que solo se recibirán entregas hasta las 23:59 del día de entrega **a menos que se especifique lo contrario**. Entregas fuera del plazo serán calificadas con nota 0.
6. Antes de entregar su laboratorio verifique su **reproducibilidad**. Jupyter Notebooks con errores a la hora de ejecutarse serán penalizados con descuentos.
7. Solo un integrante por grupo debe realizar la entrega por Aula.

<div id='experience' />

## Experiencia

En el presente laboratorio experimentaremos con las **pruebas de hipótesis**. Se comenzará evidenciando las implicancias del **teorema del límite central**, luego se diseñarán pruebas de hipótesis clásicas considerando muestras grandes y pequeñas. Finalmente, se presentará la **prueba de permutación** la cual ofrece un esquema para decidir si dos muestras provienen o no de la misma distribución **sin importar la distribución subyacente de los datos**. 

### 0. Importación de librerías

In [ ]:
import numpy as np
import pandas as pd
import math
import itertools
import matplotlib.pyplot as plt
import scipy.stats as sp

### 1. Funciones útiles

In [ ]:
def get_sample(X,parameters,seed,size=1000):
    X = X(*parameters)
    sample = X.rvs(size,random_state=seed)
    return sample

In [ ]:
def plot_sample(sample,title="",bins=100,figsize=(10,4),xlabel="x",ylabel="frecuencia"):
    fig = plt.figure(figsize=figsize)
    ax = plt.axes()
    ax.hist(sample,bins=bins)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.grid(color='gray', linestyle='solid')
    plt.show()

In [ ]:
def plot_scatter(x,y,title="",figsize=(10,4),xlabel="x",ylabel="y"):
    fig = plt.figure(figsize=figsize)
    ax = plt.axes()
    ax.plot(x,y,'-ok')
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.grid(color='gray', linestyle='solid')
    plt.show() 

<div id='tcl' />

### 2. Teorema del límite central (15 pts.)

Sea $\{X_i\}$ una muestra independiente e identicamente distribuida donde $\mu = E[X_i]$  y $ \sigma^2 = V[X_i]$. El teorema del límite central plantea que el promedio muestral $\overline{X_n}$ converge a una distribucion $N(\mu, \frac{\sigma^2}{n})$ a medida que $n \rightarrow \infty$. Esto es notable porque no es necesario ningún supuesto sobre $X_i$ más que la existencia de su media y varianza. La utilidad más evidente de este teorema es que declaraciones de probabilidad sobre $\overline{X}_n$ pueden ser aproximadas utilizando una distribución normal.

1.1) Elija su variable aleatoria favorita **(0 pts)**:

In [ ]:
def get_random_variable(option):
    if(option == 1):
        return sp.logistic()
    elif(option == 2):
        return sp.t(21)
    elif(option == 3):
        return sp.pareto(21)
    else:
        raise ValueError(f"option parameter should be between 1 and 3")

**Respuesta:**

In [ ]:
option = ...
X = get_random_variable(option)

1.2) Compruebe experimentalmente que el promedio empírico $\overline{X}$ de muestras generadas con $X$ convergen a $E[X]$ a medida que el tamaño de la muestra aumenta **(5 pts.)**

**Respuesta:**

1.3) Compruebe experimentalmente que la varianza $S^2$ del promedio empírico $\overline{X}$ de muestras generadas con $X$ convergen a $\frac{V[X]}{n}$ a medida que el tamaño de la muestra aumenta **(5 pts.)**

**Respuesta:**

1.4) Grafique cómo la distribución del promedio empírico $\overline{X}$ de muestras generadas con $X$ convergen a una distribución $N(E[X], \frac{V[X]}{n})$ a medida que el tamaño de la muestra aumenta **(5 pts.)**

> Sus visualizaciones deben hablar por sí solas! utilice títulos y nombre adecuados en los ejes.

**Respuesta:**

<div id='hp' />

### 2. Prueba de hipótesis (50 pts.)

La prueba de hipótesis es un área de la Inferencia Estadística que plantea una serie de métodos para decidir si una afirmación conocida ($H_0$) sobre una población se puede considerar aún plausible bajo la presencia de nuevos datos. 

> "Hypothesis testing is like a legal trial. We assume someone is innocent unless the evidence strongly suggests that he is guilty. Similarly, we retain $H_0$ unless there is strong evidence to reject $H_0$."


2.1) Defina brevemente los siguientes elementos fundamentales de una prueba de hipótesis **(10 pts)**:
1. Hipótesis nula $H_0$:
2. Hipótesis alternativa $H_1$:
3. Estadístico de prueba:
4. P-Value:
5. Error de tipo 1:
6. Error de tipo 2:
7. Región crítica:
**Respuesta:**

2.2) Defina la función ``hypothesis_test(sample,null_value,alpha)`` que realiza una prueba de hipótesis considerando la muestra ``sample`` (definida en la siguiente celda) y las hipótesis $H_0: \mu = 20.5$ y $H_1: \mu \neq 20.5$ considerando un nivel de significancia $\alpha = 0.05$ y el promedio muestral como estadístico de prueba **(10 pts.)**

> Considere como **valor de retorno** un valor booleano que indica si H0 se rechaza o no.

**Respuesta:**

In [ ]:
sample = get_sample(sp.norm,[21,5],seed=21,size=100)

In [ ]:
def hypothesis_test(sample,null_value,alpha):
    
    # Get sample statistics
    n = ...
    sample_mean = ...
    sample_std = ...
    
    # Get z (value of test statistic)
    z = ...
    
    # Get P-value
    Z = ...
    P_value =  ...
    
    # Show P-value
    print(f"P-value is: {P_value}")
    
    # H0 is rejected?
    print("H0 is rejected") if ... else print("H0 is not rejected")
    
    return ...

In [ ]:
hypothesis_test(sample,20.5,0.05)

2.3) Comente los resultados de la prueba de hipótesis realizada en 2.2, además, interprete el P-value obtenido **(5 pts.)**

**Respuesta:** 

2.4) ¿Qué ocurre con el resultado de la prueba de hipótesis realizada en 2.2 a medida que el tamaño de la muestra aumenta?, ¿por qué? **(5 pts.)**

**Respuesta:**

2.5) Defina la función ``get_threshold(sample,null_value,alpha)`` que retorna el valor límite (valor crítico) para el promedio muestral que permite considerar $H_0$ aún plausible en base a la muestra ``sample``, el valor nulo ``null_value`` y el nivel de significancia ``alpha`` **(10 pts.)**

La siguiente imagen puede ser de ayuda, aunque debe considerar que es este caso estamos realizando una prueba de hipótesis "two-sided":

<center>
    <img src="errors.png" style="width:60%">
</center>

**Respuesta:**

In [ ]:
def get_threshold(sample,null_value,alpha):
    
    # Get sample statistics
    n = ...
    sample_mean = ...
    sample_std = ...
    
    # Define N
    N = ...
    
    # Get threshold
    threshold = ...
    print(f"Threshold: {threshold}")
    
    return threshold

In [ ]:
threshold = get_threshold(sample,20.5,0.05)

In [ ]:
#hypothesis_test(sample,threshold,0.05)

2.5) Cuando el tamaño de la muestra es pequeño sabemos que el teorema del límite central es impreciso, lo que implica que nuestra prueba de hipótesis lo es también. Defina la función ``hypothesis_test_2(sample,null_value,alpha)`` que realiza una prueba de hipótesis igual a la realizada en la pregunta 2.2 pero que permita evitar el problema mencionado respecto a las muestras pequeñas **(10 pts.)**

> Considere como **valor de retorno** un valor booleano que indica si H0 se rechaza o no.

In [ ]:
def hypothesis_test_2(sample,null_value,alpha):
    
    ...
    
    return ...

In [ ]:
hypothesis_test_2(sample,20.5,0.05)

<div id='pt' />

### 3. Prueba de permutación (35 pts.)

La prueba de permutación (permutation test) es un método no parámetrico para probar si dos muestras distintas provienen de la misma distribución de probabilidad. Dadas dos muestras independientes $\{X_i\}$ e $\{Y_j\}$ distribuidas respectivamente por $F_X$ e $F_Y$ y la hipótesis nula $H_0$: $F_X = F_Y$ el procedimento para una prueba de permutación considerando un estadístico de prueba $T(X_1,\dots,X_{m-1},X_m,Y_1,\dots,Y_{n-1}, Y_n)$ queda descrito a continuación:

1. Calcular el valor observado del estadístico de prueba: $t = T(X_1,\dots,X_{m-1},X_m,Y_1,\dots,Y_{n-1}, Y_n)$
2. Computar las $N! = (m + n)!$ permutaciones del vector que contiene los datos de ambas muestras $(X_1,\dots,X_{m-1},X_m,Y_1,\dots,Y_{n-1}, Y_n)$
3. Calcular el estadístico de prueba para cada una de las $N!$ permutaciones.
4. Calcular el P-value $P(T > t) = \frac{1}{N!} \sum_{i = 1}^{N!} I(T_i > t)$.
5. Decidir en base a P-value y alpha.

3.1) Defina la función ``permutation_test(X,Y,alpha)`` que realiza una prueba de permutación considerando las muestras ``X`` e ``Y``, un nivel de confianza ``alpha`` y el estadístico de prueba $T(X,Y) = |\overline{X} - \overline{Y}|$ **(15 pts.)**

> Considere como **valor de retorno** un valor booleano que indica si H0 se rechaza o no.

**Respuesta:**

In [ ]:
def permutation_test(X,Y,alpha):
    
    # Get sample sizes
    m = ...
    n = ...
    
    # Concatenate samples
    sample = ...
    
    # Step 1: Get t (value of test statistic)
    t = ...
    
    # Step 2: Get N! permutations
    permutations = ...
    
    # Step 3: Get sample statistic for every permutation
    sample_statistic_values = ...
    
    # Step 4: Get P-Value
    P_value = ...
    
    # Show P-value
    print(f"P-value is: {P_value}")
    
    # Step 5: Decide
    # H0 is rejected?
    print("H0 is rejected") if ... else print("H0 is not rejected")
    
    return ...

3.2) Realice una prueba de permutación considerando las muestras ``sample_x`` y ``sample_y`` definidas en la siguiente celda. Comente su resultado e interprete el P-value **(5 pts.)**

In [ ]:
sample_x = list(sp.poisson(100).rvs(4,random_state=11))
sample_y = list(sp.poisson(80).rvs(4,random_state=11))

**Respuesta:**

In [ ]:
permutation_test(sample_x,sample_y,0.05)

3.3) Evidentemente la generación de $N!$ permutaciones es extremadamente costoso, por ejemplo si cada muestra consiste de 25 elementos se necesitarían generar $50! = 30414093201713378043612608166064768844377641568960512000000000000$ permutaciones. Plantee un algoritmo (paso a paso) que permita realizar una prueba de permutación evitando este problema **(5 pts.)**

**Respuesta:**

3.4) Defina la función ``permutation_test_2(...)`` que realiza la prueba de permutación utilizando su algoritmo planteado en la pregunta 3.3  **(10 pts.)**

> Considere como **valor de retorno** un valor booleano que indica si H0 se rechaza o no.

**Respuesta:**

In [ ]:
def permutation_test_2(X,Y,alpha,size):
    ...
    return ...

In [ ]:
permutation_test_2(sample_x,sample_y,0.05,1000)

## The end of the road (or the beginning of a new one) 

In [ ]:
print("""
Felicitaciones ! Acabas de completar el cuarto y último LEC. Espero haya sido una enriquecedora experiencia
de aprendizaje :') 
""")

En esta ocasión, el laboratorio será corregido **rigurosamente** por la siguiente función : 

In [ ]:
def get_grade():
    return sp.uniform(0,100).rvs()

In [ ]:
grade = get_grade()

In [ ]:
print(f"Que buena noticia! obtuviste un {grade}.")